In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random 

import xgboost as xgb

from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

rng = np.random.RandomState(31337)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

df = pd.read_csv('../input/train.csv')

df.describe()

In [ ]:
X = df.drop('price_doc', axis=1).select_dtypes(exclude=[object])
y = df['price_doc']

In [ ]:
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    xgb_model = xgb.XGBRegressor().fit(X.iloc[train_index],y[train_index])
    predictions = xgb_model.predict(X.iloc[test_index])
    actuals = y[test_index]
    print(mean_squared_error(actuals, predictions))

In [ ]:
xgb_model = xgb.XGBRegressor()
clf = GridSearchCV(xgb_model,
                   {'max_depth': [2,4,6],
                    'n_estimators': [50,100,200]}, verbose=1)
clf.fit(X,y)
print(clf.best_score_)
print(clf.best_params_)

In [ ]:
test = pd.read_csv('../input/test.csv')

In [ ]:
obj_cols = list(df.select_dtypes(include=[object]).drop('timestamp', axis=1).columns)
obj_cols

In [ ]:
dummy_train = pd.get_dummies(df[obj_cols])
dummy_test = pd.get_dummies(test[obj_cols])

In [ ]:
X = pd.concat([X, dummy_train], axis=1)
X_test = pd.concat([test, dummy_test], axis=1).select_dtypes(exclude=[object]).

In [ ]:
params = {'max_depth': 4, 'n_estimators': 200}

In [ ]:
xgb_reg = xgb.XGBRegressor(**params).fit(X.drop('id', axis=1), y)
predictions = xgb_reg.predict(X_test.drop('id', axis=1).as_matrix())

In [ ]:
boosting_sub = pd.DataFrame({"id":X_test["id"], "price_doc":predictions})
boosting_sub.to_csv("boosting.csv", index=False)